In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
from langgraph.graph import StateGraph, END 
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import  AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

# Define tool that is available to langraph that an action edge can find
tool = TavilySearchResults(max_results=3)
print(f"tool name -> {tool.name}")


tool name -> tavily_search_results_json


/var/folders/8b/hyz6y59163s8qzgrs9d1r7fw0000gn/T/ipykernel_25391/1418715925.py:11: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(max_results=3)


#### Define A place holder for all the messages that is known as AgentState
- This is a list of messages which keeps adding a new message everytime it is called

In [2]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

#### Define the Agent Class that performs:
1. Call LLM in this example OpenAI
2. Check if Action is present
3. Take Action

Steps in the code 
1. The constructor function takes model name, available tools choices and system prompt
2. Start a LLM node
3. Then add an action node
4. Then define an action edge to link between LLM and action node. 
5. If no action decision made by LLM, then send to END node
6. Create an edge to loop back to LLM node from Action Node
7. Compile the graph and save it as class level attribute
8. Create a dictionary of tools sent as parameters and save as class level attribute
9. Save the tool name that sent as input as a class level attribute under the model

In [3]:
class Agent:
    def __init__(self,model,tools,system=""):
        # Save the system message as a class level attribute
        self.system = system

        # Initialize the state graph that will have one LLM node, One Tool node and one Action Edge
        graph = StateGraph(AgentState)
        # Start Node
        graph.add_node("llm",self.call_openai)
        
        # Action Node that is available as tool
        graph.add_node("action",self.action_node)

        # Decision Edge to decide to use action node
        # First parameter is the node from which this edge is coming from 
        # Second parameter is the function that let's langraph explore tools
        # Third parameter is available nodes after the decision either action node or END node
        graph.add_conditional_edges("llm",self.action_edge,{True: "action", False: END})

        # Create Another edge to loop back to LLM node from action node
        graph.add_edge("action","llm")

        # Define what node the graph should start, in this case the llm
        graph.set_entry_point("llm")

        # Once setup done compile the graph and Save the graph at the class level
        self.graph = graph.compile()

        # Create a dictionary of available tools sent to the constructor
        self.tools = {tool.name: tool for tool in tools}

        # Bind tools to model so that LLM can search for tools
        self.model = model.bind_tools(tools)


    # Define function for call llm node
    def call_openai(self, state: AgentState):
        # get the messages saved in the Agent state object
        messages = state["messages"]
        # If system message is not blank, append that to the beginning of the messages
        if self.system:
            system_message= [SystemMessage(content=self.system)]
            messages =  system_message + messages
        # Call the model with the messages, it should return response as a single message
        resp = self.model.invoke(messages)
        print(f"Response from LLM -> {resp}")
        # Return the response message as a list, that will be appended to the existing messages due to operator.add annotation at class level
        return {"messages": [resp]}
    
    # Define function for call action node
    def action_node(self, state: AgentState):
        # get the last message from the Agent State, since the last message is the response from LLM that suggests to use the tool
        # tool calls attribute is expected which has the name of the tool to be called
        referred_tools_list = state["messages"][-1].tool_calls
        results = []

        # Tool calls can be multiple tools, so iterate over them
        for tool in referred_tools_list:
            print(f"Tool to be called -> {tool}")
            # invoke tool call by finding the name and the arguments as suggested by LLM
            result = self.tools[tool["name"]].invoke(tool["args"])
            # Append the result to the results list
            results.append(ToolMessage(tool_call_id=tool["id"], name=tool["name"], content=str(result)))
            
        print(f"Finished tool call ...")
        # returns results and add to the messages list at class level
        return {"messages": results}
    
    # Define the actiton edge function that decides whether to look for tool or not
    # If the last message in the message list has tool_calls attribute, then return True, else False
    def action_edge(self, state: AgentState):
        result = state["messages"][-1]
        return  len(result.tool_calls) > 0
        
     

#### Define a chat model with system prompt

In [4]:
system_prompt = prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")
ai_agent = Agent(model,[tool],system_prompt)

#### Invoke the langraph with user message as input

In [ ]:
user_prompt = "What is the capital of France?"
messages = [HumanMessage(content=user_prompt)]
result = ai_agent.graph.invoke({"messages": messages})
print(result["messages"][-1].content)